# 2.4.3. Модель word2vec и сервис RusVectōrēs.  

Видеоуроки:  
2.4.1. Векторное представление единиц текста.  
2.4.2. Оценка близости двух текстов.  
**2.4.3. Модель word2vec сервиса RusVectōrēs.**  

Дополнительные материалы:  
2.4.4. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели FastText сервиса RusVectōrēs".  
2.4.5. Скринкаст "Оценка семантической близости вопросов и ответов с использованием модели Elmo сервиса RusVectōrēs".  

> "A word is characterized by the company it keeps"  
[John Rupert Firth](https://en.wikipedia.org/wiki/John_Rupert_Firth)  

**Дистрибутивная гипотеза** - лингвистические единицы, встречающиеся в схожих контекстах, имеют близкие значения (значение слова определяется словами, с которыми оно употребляется).  
Например слова "чай" и "кофе" семантически близки так как употребляются вместе со словами "ложка", "чашка".  
  
**Языковая модель (статистическая)** - распределение вероятностей по последовательностям слов. Нейросетевые модели, обученные предсказывать слово с учетом контекста. Например: "Мама мыла Х", где слово "раму" имеет вероятность больше, чем слово "папу".  
  
**Word2Vec (word to vector)** - нейросетевая языковая модель обучения без учителя, получает на вход слово, на выходе - вектор фиксированной размерности. Задача модели - представить слова таким вектором, чтобы слова со схожим смыслом (контекстом) были близки, а с различающимся - далеки друг от друга.  
Word2Vec обучается на больших объемах текстов (текстах новостей, википедии), что позволяет получить обширные "знания" (языковую интуицию). В результате модель можно переиспользовать на разных задачах.  
  
**Embedding ([вложение](https://ru.wikipedia.org/wiki/Вложение))** - векторное представление слова/текста (результат на выходе из модели).  

Сервис [RusVectōrēs](https://rusvectores.org/ru/about/) вычисляет семантические отношения между словами русского языка и позволяет скачать предобученные дистрибутивно-семантические модели (word embeddings).  
  
> Обучение дистрибутивных моделей на основе больших корпусов может требовать существенных вычислительных мощностей. Поэтому важно предоставить русскоязычному лингвистическому сообществу доступ к предобученным моделям. Наш сервис дает пользователям готовые модели для скачивания (чтобы продолжить эксперименты на своём компьютере), а также удобный интерфейс запросов к ним. Также возможно визуализировать семантические отношения между словами, что, как мы надеемся, будет полезным для исследователей. В целом, задача нашего сервиса — снизить порог входа для тех, кто хочет работать в этом новом и интересном направлении.
  
Выбрав модель, вы можете:

1. вычислять семантическое сходство между парами слов;  
2. находить слова, ближайшие к данному (с возможностью фильтрации по части речи и частотности);  
3. решать аналогии вида «найти слово X, которое так относится к слову Y, как слово A относится к слову B»;  
4. выполнять над векторами слов алгебраические операции (сложение, вычитание, поиск центра лексического кластера и расстояний до этого центра).
5. рисовать семантические карты отношений между словами (это позволяет выявлять семантические кластеры или тестировать ваши гипотезы о таких кластерах);  
6. получать вектор (в виде массива чисел) и его визуализацию для данного слова в выбранной модели: для этого нужно кликнуть по любому слову или использовать уникальный адрес этого слова, как описано ниже;  
7. генерировать контекстно-зависимые лексические подстановки для контекстуализированных дистрибутивных моделей, например, ELMo;  
8. скачать модель.  
  
Видео - https://youtu.be/wWG204boB5s  

## Использование моделей RusVectōrēs.  
  
В основе RusVectores лежит библиотека [Gensim](https://radimrehurek.com/gensim/).  
Gensim - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для topic modeling в ней реализованы на python и алгоритмы word2vec (который в оригинале был написан на C++).  

In [11]:
from typing import List
import gensim, logging
import zipfile
import wget
import warnings

# Уберем лишние предупреждения
warnings.filterwarnings('ignore')
# Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий.
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Если у вас есть достаточно большой корпус текстов - вы можете натренировать модель самостоятельно на своих данных. Но для общих целей есть смысл использовать готовые модели, натренированные на больших объемах текстов как русского так и английского языков. 

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

### Word2Vec
Давайте скачаем модель Word2Vec для русского языка, созданную на основе [Национального Корпуса Русского Языка (НКРЯ)](http://www.ruscorpora.ru/), и загрузим в её в память. Распаковывать скачанный архив для обычных моделей не нужно, так как его содержимое прочитается при помощи специальной инструкции:

In [2]:
MODEL_W2V_URL = 'http://vectors.nlpl.eu/repository/11/180.zip'

# размер файла - 460 Mb.
# Закомментируйте строку, если скачивали файл модели ранее.
# m = wget.download(MODEL_W2V_URL)
model_file = MODEL_W2V_URL.split('/')[-1]

with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model_w2v = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

2022-01-19 15:54:41,906 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2022-01-19 15:54:44,926 : INFO : loaded (189193, 300) matrix from <zipfile.ZipExtFile [closed]>


Проанализируем несколько слов.  
В этой модели для каждого слова указывается часть речи. Подробнее о тегах частей речи можно почитать тут - https://universaldependencies.org/u/pos/all.html  
`NOUN` - существительное,  
`ADJ` - прилагательное,  
`VERB` - глагол.  

In [42]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

Запросим у модели 5 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [47]:
def print_neighbors(words: List[str], model: gensim.models.KeyedVectors, top_n: int = 5):
    """
    Печатаем семантически близкие слова.

    Args:
        words (List[str]): список слов, для которых нужно найти соседей.
        model (gensim.models.KeyedVectors): модель gensim.
        top_n (int): количество соседей. По умолчанию 5.
    """

    col_width = 20
    print(*[c.ljust(col_width) for c in ['Слово', 'Сосед', 'Коэффициент косинуссной близости']])
    print('-'*col_width*3)

    for word in words:
        # есть ли слово в модели? Может быть, и нет.
        if word in model:
            # выдаем top_n ближайших соседей слова:
            for i in model.most_similar(positive=[word], topn=top_n):
                w2 = i[0]
                d = round(i[1], 3)
                print(word.ljust(col_width), w2.ljust(col_width), d)
            print('')
        else:
            # Out of vocabulary
            print(word + ' - OOV! 😒')

In [48]:
print_neighbors(words, model_w2v, 5)

Слово                Сосед                Коэффициент косинуссной близости
------------------------------------------------------------
день_NOUN            неделя_NOUN          0.738
день_NOUN            день_PROPN           0.707
день_NOUN            месяц_NOUN           0.704
день_NOUN            час_NOUN             0.664
день_NOUN            утро_NOUN            0.653

ночь_NOUN            ночь_PROPN           0.831
ночь_NOUN            вечер_NOUN           0.718
ночь_NOUN            рассвет_NOUN         0.697
ночь_NOUN            ночи_NOUN            0.692
ночь_NOUN            полночь_NOUN         0.67

человек_NOUN         человек_PROPN        0.785
человек_NOUN         человеческий_ADJ     0.592
человек_NOUN         существо_NOUN        0.574
человек_NOUN         народ_NOUN           0.535
человек_NOUN         личность_NOUN        0.53

семантика_NOUN       семантический_ADJ    0.802
семантика_NOUN       синтаксический_ADJ   0.757
семантика_NOUN       модальный_ADJ        0.73


Word2Vec не может обрабатывать слова, которых нет в словаре.  
Давайте посмотрим, какие слова есть в словаре модели:  

In [17]:
model_w2v.index2word[:10]

['так_ADV',
 'быть_VERB',
 'мочь_VERB',
 'год_NOUN',
 'человек_NOUN',
 'xxxxxx_NUM',
 'сказать_VERB',
 'еще_ADV',
 'один_NUM',
 'говорить_VERB']

Косинусная близость пары слов.

In [18]:
print(model_w2v.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.22025342


Найди лишнее слово.

In [19]:
print(model_w2v.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

картофель_NOUN


In [20]:
print(model_w2v.doesnt_match('сельдь_NOUN италия_NOUN германия_NOUN россия_NOUN'.split()))

2022-01-19 16:02:17,526 : WARNING : vectors for words {'германия_NOUN'} are not present in the model, ignoring these words


сельдь_NOUN


Борщ + Россия - Италия

In [21]:
print(model_w2v.most_similar(positive=['борщ_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

суп_NOUN


In [22]:
print(model_w2v.most_similar(positive=['кофеин_NOUN', 'кофе_NOUN'], negative=['чай_NOUN'])[0][0])

антиоксидант_NOUN


## Инструменты на сайте RusVectōrēs.  
На сайте проекта [RusVectōrēs](https://rusvectores.org) доступны различные инструменты, позволяющие проводиь эксперименты с моделями без использования программного кода.  

![alt text](https://rusvectores.org/data/images/ruwikiruscorpora_1k_nouns.png)


# Выводы 

1. Сервис RusVectōrēs предоставляет доступ к большому количеству предобученных моделей.  
2. Модели RusVectores позволяют решать несколько задач:  
        - вычислять семантическое сходство между парами слов;  
        - находить слова, ближайшие к данному;  
        - решать аналогии вида «найти слово X, которое так относится к слову Y, как слово A относится к слову B»;  
        - и прочее.  
3. Использование готовой модели, обученной на большом тексте, дает много плюсов для использования в своих проектах (примеры будут рассмотрены в следующих темах).  
4. Много инструментов для экспериментов доступны на сайте проекта.  

# Дополнительные материалы:  
  
https://rusvectores.org  
https://radimrehurek.com/gensim  
https://fasttext.cc  
